# Proyecto 2
## Security Data Science
* César Rodas 16776
* Mario Andrés Perdomo 18029

In [1]:
import pandas as pd
import numpy as np
%matplotlib agg
import matplotlib.pyplot as plt
import seaborn as sns

#!{sys.executable} -m pip install pandas_profiling
!pip install pandas_profiling
!pip install sklearn
import pandas_profiling as pp
from pandas_profiling import ProfileReport

import sklearn
from sklearn import metrics, model_selection, tree

from urllib.parse import urlparse

You should consider upgrading via the 'c:\users\mario\appdata\local\programs\python\python39\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'c:\users\mario\appdata\local\programs\python\python39\python.exe -m pip install --upgrade pip' command.


### Separación y Optimización de datos

In [2]:
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))

    return df

In [3]:
df = pd.read_csv('clean_train.csv')

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8588801 entries, 0 to 8588800
Data columns (total 50 columns):
 #   Column                                             Dtype
---  ------                                             -----
 0   ProductName                                        int64
 1   IsBeta                                             int64
 2   RtpStateBitfield                                   int64
 3   IsSxsPassiveMode                                   int64
 4   AVProductStatesIdentifier                          int64
 5   AVProductsInstalled                                int64
 6   AVProductsEnabled                                  int64
 7   HasTpm                                             int64
 8   CountryIdentifier                                  int64
 9   GeoNameIdentifier                                  int64
 10  LocaleEnglishNameIdentifier                        int64
 11  Platform                                           int64
 12  Processor     

In [5]:
df.head(10)

,ProductName,IsBeta,RtpStateBitfield,IsSxsPassiveMode,AVProductStatesIdentifier,AVProductsInstalled,AVProductsEnabled,HasTpm,CountryIdentifier,GeoNameIdentifier,...,Census_IsPortableOperatingSystem,Census_GenuineStateName,Census_ActivationChannel,Census_FlightRing,Census_IsSecureBootEnabled,Census_IsVirtualDevice,Census_IsTouchEnabled,Census_IsPenCapable,Census_IsAlwaysOnAlwaysConnectedCapable,HasDetections
0,0,0,7,0,53447,1,1,1,29,35,...,0,0,0,0,0,0,0,0,0,0
1,0,0,7,0,53447,1,1,1,93,119,...,0,2,0,1,0,0,0,0,0,0
2,0,0,7,0,53447,1,1,1,86,64,...,0,0,3,0,0,0,0,0,0,0
3,0,0,7,0,53447,1,1,1,88,117,...,0,0,3,0,0,0,0,0,0,1
4,0,0,7,0,53447,1,1,1,18,277,...,0,0,0,0,0,0,0,0,0,1
5,0,0,7,0,53447,1,1,1,97,126,...,0,0,0,0,0,0,0,0,0,1
6,0,0,7,0,43927,2,1,1,78,89,...,0,0,0,0,1,0,0,0,0,1
7,0,0,7,0,53447,1,1,1,97,126,...,0,0,0,0,0,0,0,0,0,0
8,0,0,7,0,53447,1,1,1,164,205,...,0,0,0,0,0,0,0,0,0,0
9,0,0,7,0,53447,1,1,1,107,138,...,0,0,0,0,0,0,1,1,1,0


In [6]:
df = reduce_mem_usage(df)

Memory usage after optimization is: 655.27 MB
Decreased by 80.0%


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8588801 entries, 0 to 8588800
Data columns (total 50 columns):
 #   Column                                             Dtype
---  ------                                             -----
 0   ProductName                                        int8 
 1   IsBeta                                             int8 
 2   RtpStateBitfield                                   int8 
 3   IsSxsPassiveMode                                   int8 
 4   AVProductStatesIdentifier                          int32
 5   AVProductsInstalled                                int8 
 6   AVProductsEnabled                                  int8 
 7   HasTpm                                             int8 
 8   CountryIdentifier                                  int16
 9   GeoNameIdentifier                                  int16
 10  LocaleEnglishNameIdentifier                        int16
 11  Platform                                           int8 
 12  Processor     

In [8]:
target = df['HasDetections']
feature = df.drop(['HasDetections'], axis=1)

** Separamos en dos el dataset train 70% test 30%**

In [9]:
feature_train, feature_test, target_train, target_test = model_selection.train_test_split(feature, target, test_size=0.3, random_state=1)

** Separamos nuevamente el train para obtener train 55% validacion 15%**

In [10]:
feature_train, feature_validate, target_train, target_validate = model_selection.train_test_split(feature_train, target_train, test_size=0.15, random_state=1)

In [11]:
feature_train.count()

ProductName                                          5110336
IsBeta                                               5110336
RtpStateBitfield                                     5110336
IsSxsPassiveMode                                     5110336
AVProductStatesIdentifier                            5110336
AVProductsInstalled                                  5110336
AVProductsEnabled                                    5110336
HasTpm                                               5110336
CountryIdentifier                                    5110336
GeoNameIdentifier                                    5110336
LocaleEnglishNameIdentifier                          5110336
Platform                                             5110336
Processor                                            5110336
OsBuild                                              5110336
OsSuite                                              5110336
OsPlatformSubRelease                                 5110336
SkuEdition              

In [12]:
feature_test.count()

ProductName                                          2576641
IsBeta                                               2576641
RtpStateBitfield                                     2576641
IsSxsPassiveMode                                     2576641
AVProductStatesIdentifier                            2576641
AVProductsInstalled                                  2576641
AVProductsEnabled                                    2576641
HasTpm                                               2576641
CountryIdentifier                                    2576641
GeoNameIdentifier                                    2576641
LocaleEnglishNameIdentifier                          2576641
Platform                                             2576641
Processor                                            2576641
OsBuild                                              2576641
OsSuite                                              2576641
OsPlatformSubRelease                                 2576641
SkuEdition              

In [13]:
feature_validate.count()

ProductName                                          901824
IsBeta                                               901824
RtpStateBitfield                                     901824
IsSxsPassiveMode                                     901824
AVProductStatesIdentifier                            901824
AVProductsInstalled                                  901824
AVProductsEnabled                                    901824
HasTpm                                               901824
CountryIdentifier                                    901824
GeoNameIdentifier                                    901824
LocaleEnglishNameIdentifier                          901824
Platform                                             901824
Processor                                            901824
OsBuild                                              901824
OsSuite                                              901824
OsPlatformSubRelease                                 901824
SkuEdition                              

## MODELO

In [14]:
from sklearn.metrics import roc_curve, auc, roc_auc_score
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

In [15]:
from sklearn.naive_bayes import GaussianNB

In [21]:
from sklearn.neighbors import KNeighborsClassifier

### Implementación de Naive Bayes

In [26]:
gnb = KNeighborsClassifier()

In [27]:
gnb.fit(feature_train,target_train)

KNeighborsClassifier()

In [ ]:
fetaure_pred=gnb.predict(feature_test)

In [ ]:
print('accuracy score \n',accuracy_score(target_test, fetaure_pred))

In [20]:
print('Matriz de confusion \n',confusion_matrix(target_test, target_pred))

NameError: name 'target_pred' is not defined

In [24]:
print(classification_report(target_test, fetaure_pred, target_names=target_names))

NameError: name 'target_names' is not defined

In [25]:
target_prob = clf.predict_proba(feature_test)      

NameError: name 'clf' is not defined

## Curva de Roc

In [ ]:
plt.style.use('seaborn')
%matplotlib inline
# roc curve for classes
fpr = {}
tpr = {}
thresh ={}

n_class = 2

for i in range(n_class):    
    fpr[i], tpr[i], thresh[i] = roc_curve(target_test, target_prob[:, i], pos_label=i)
    
# plotting    
plt.plot(fpr[0], tpr[0], linestyle='--',color='orange', label='Class 0 vs Result')
plt.plot(fpr[1], tpr[1], linestyle='--',color='green', label='Class 1 vs Result')

plt.title('Multiclass ROC curve')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive rate')
plt.legend(loc='best')
#plt.savefig('LR Multiclass ROC',dpi=900); 

## Kfolds 

In [ ]:
from sklearn.model_selection import KFold

In [ ]:
cv = KFold(n_splits=10, random_state=1, shuffle=True)

In [ ]:
#use k-fold CV to evaluate model
scores = cross_val_score(clf, X, y, scoring='neg_mean_absolute_error',
                         cv=cv, n_jobs=-1)

In [ ]:
from numpy import mean
mean(absolute(scores))